In [ ]:
'''
Return variables dictionary from JSON url 
'''
def read_census_json(json_url): 
    
    # Make a GET request to the JSON url 
    response = requests.get(json_url)
    
    # Check if it was successful (status code == 200)
    if response.status_code == 200: 
        # This is the initial response, in dictionary form 
        json_dict = response.json()
        # There is only one key, so reassign the dictionary variable to that key 
        variables_dict = json_dict['variables']
    else: 
        print("Couldn't read JSON")
    
    print("JSON dictionary returned")
    
    return variables_dict

In [ ]:
'''
Rename columns
'''
def label_index_values(json_list, code_label_dictionary): 
    
    index_list = json_list[0][0]

    for i, index in enumerate(index_list): 

        try: 
            index_list[i] = code_label_dictionary[index]
        except: 
            pass 
    
    return index_list 

In [ ]:
'''
Convert JSON to dictionary
'''
def json_to_dictionary(json_list, index_list): 

    output_dictionary = {} 

    for element in json_list: 
        if element[1] == index_list: 
            for index in index_list: 
                output_dictionary[index] = [] 

        else: 
            for i, variable in enumerate(element[1]): 
                output_dictionary[index_list[i]].append(variable)

    return output_dictionary

In [ ]:
#################################################################################################################

In [ ]:
'''
Converts output of API calls to dataframe. Index is created from variables, rows each represent a block group. 
'''

def json_list_to_dataframe(json_list):

    output_dictionary = {} 
    
    index_list = json_list[0][0]
    
    for index in index_list: 
        output_dictionary[index] = []

    for json in json_list: 
        values = json[1]

        for i, value in enumerate(values): 
            output_dictionary[index_list[i]].append(value)

    api_dataframe = pd.DataFrame.from_dict(output_dictionary)
    api_dataframe = api_dataframe.rename(columns = code_label_dictionary)
    
    api_dataframe['FIPS'] = api_dataframe['state'] + api_dataframe['county'] + api_dataframe['tract'] + api_dataframe['block group']
    
    return api_dataframe 

In [ ]:
'''
Change columns to integers if value is numeric. 
'''
def change_columns_to_integers(dataframe): 
    
    for column in dataframe.columns: 

        try: 
            dataframe[column] = dataframe[column].astype(int)
        except: 
            pass
            
    return dataframe 

In [ ]:
'''
Calculate percentage columns. 
'''
def calculate_percentage_columns(dataframe):
    
    # Iterate through columns 
    for column in dataframe.columns:
        
        # If it is in the dictionary
        if column in code_label_dictionary.values(): 
            
            # Identify the column's position  
            column_position = dataframe.columns.get_loc(column) + 1
            # Calculate its value
            percentage_formula = (dataframe[column]/dataframe['Total Population'])*100
            # Insert the new column 
            dataframe.insert(column_position, f'% {column}', round(percentage_formula, 2))
            
    return dataframe 

In [ ]:
'''
Master Function 
'''

def create_api_dataframe(json_list): 

    api_dataframe = json_list_to_dataframe(json_list)
    api_dataframe = change_columns_to_integers(api_dataframe)
    api_datafrmae = calculate_percentage_columns(api_dataframe)
    
    return api_dataframe

'''
Parameters
'''
json_list

'''
Run
'''
api_dataframe = create_api_dataframe(json_list)

api_dataframe